<h2> pyBess Example Notebook </h2>

<h4> Import Modules </h4>

In [1]:
import pandas as pd
import datetime
from pybess.Battery import Battery
from pybess.Arbitrage import Arbitrage
from pybess.utils import *
from IPython.core.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

C:\Users\Henry\Anaconda3\envs\thesisenv\lib\site-packages\nemosis\custom_tables.py


<h4> Define Battery </h4>

In [2]:
battery = Battery(cap_power=30, cap_store=119, charge_eff=0.9, dcharge_factor=1/0.9, cap_init=0)

<h4> Define Constants </h4> 
<p> Note that the start-date must be greater than 2010-01-01 and the end date cannot be equal or greater than today's date. </p>

In [3]:
start_date = datetime.datetime(2018, 1, 1)
end_date = datetime.datetime(2019, 1, 1)
state="SA1"

Using a historical price series, energy arbitrage can be formulated with the following maximisation function:
\begin{align*}
  \max \sum_{i=0}^T \left(x^{(i)}_c + x^{(i)}_d \right) p^{(i)} \times \left( \frac{l}{60} \right)
\end{align*}
Subject to the constraints:
\begin{alignat} {2}
    -P_{max} &\leq x^{(i)}_c \leq 0  &&\forall i \in [0,T]\\
    0 &\leq x^{(i)}_d \leq P_{max}  &&\forall i \in [0,T]\\
    0 &\leq E^{(i)} \leq E_{max} &&\forall i \in [0,T] \\
    E^{(0)} &= E_{\text{init}} - \left(x^{(0)}_c \eta_c + x^{(0)}_d \frac{1}{\eta_d} \right) \frac{l}{60} && \\
    E^{(i)} &= E^{(i-1)} - \left(x^{(i)}_c \eta_c + x^{(i)}_d \frac{1}{\eta_d} \right) \frac{l}{60} \hspace{1cm} &&\forall i \in [1,T] 
\end{alignat}

In [4]:
arb = Arbitrage(battery, start_date, end_date, state)
arb.solve()
arb_results = arb.results()

Compiling data for table TRADINGPRICE.
Creating feather file for faster future access of table TRADINGPRICE, year 2017, month 12.
Creating feather file for faster future access of table TRADINGPRICE, year 2018, month 01.
Creating feather file for faster future access of table TRADINGPRICE, year 2018, month 02.
Creating feather file for faster future access of table TRADINGPRICE, year 2018, month 03.
Creating feather file for faster future access of table TRADINGPRICE, year 2018, month 04.
Creating feather file for faster future access of table TRADINGPRICE, year 2018, month 05.
Creating feather file for faster future access of table TRADINGPRICE, year 2018, month 06.
Creating feather file for faster future access of table TRADINGPRICE, year 2018, month 07.
Creating feather file for faster future access of table TRADINGPRICE, year 2018, month 08.
Creating feather file for faster future access of table TRADINGPRICE, year 2018, month 09.
Creating feather file for faster future access of t

<h4> Explore Outputs </h4>

<h5> Explore Half Hourly Output </h5>

In [5]:
display(arb_results.head())

,timestamp,discharge,charge,dispatch,stored_energy,RRP
0,2018-01-01 00:30:00,0.0,0.0,0.0,0.0,107.17
1,2018-01-01 01:00:00,0.0,0.0,0.0,0.0,103.31
2,2018-01-01 01:30:00,0.0,0.0,0.0,0.0,88.20
3,2018-01-01 02:00:00,0.0,0.0,0.0,0.0,85.24
4,2018-01-01 02:30:00,0.0,0.0,0.0,0.0,81.75


<h5> Explore Revenue by Month </h5>

As trading price is half hourly, revenue is given by:
\begin{equation}
    \text{Revenue (\$)} =  \dfrac{Target(MW) \times RRP (\$/MWh)}{2} 
\end{equation}

In [6]:
arb_results['revenue'] = arb_results['dispatch']*arb_results['RRP']/2
arb_results['year_month'] = arb_results['timestamp'].shift(1).dt.strftime('%Y-%m-01') #Trading Interval Ending
monthly_arb_results =  arb_results.groupby('year_month')['revenue'].sum().reset_index()
monthly_arb_results_formatted = monthly_arb_results
monthly_arb_results_formatted['revenue'] = monthly_arb_results_formatted['revenue'].map('${:,.2f}'.format)
display(monthly_arb_results_formatted.head(n=12))

,year_month,revenue
0,2018-01-01,"$1,675,352.68"
1,2018-02-01,"$476,505.93"
2,2018-03-01,"$158,296.79"
3,2018-04-01,"$532,167.67"
4,2018-05-01,"$302,727.22"
5,2018-06-01,"$364,877.28"
6,2018-07-01,"$1,006,123.46"
7,2018-08-01,"$462,065.32"
8,2018-09-01,"$387,747.71"
9,2018-10-01,"$331,082.04"


<h4> Plot Ouputs </h4>

In [7]:
display(nbplot(arb))

The draw time for this plot will be slow for clients without much RAM.
